## Question 1 Financial Statistics

Import libraries and read the given excel data using panda function, store as data frame raw (without filter yet)

In [37]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# read the excel data
df_raw = pd.read_excel("data_coursework1_Q1.xls")


Selecting relevant columns and clean the raw data frame by only keeping 3 columns we need for the CAPM analysis:
1) SP500
2) IBM
3) 1-month Tbill

Then convert these columns to numeric values and coerce any non-numeric entries to drop rows containing missing values.

In [38]:
# SP500: market price/index level
# IBM: stock price
# 1-m Tbill: risk-free rate (given data is multiplied by 100 and in month)
prices = df_raw[["SP500", "IBM", "1-month Tbill"]].copy()

prices["SP500"] = pd.to_numeric(prices["SP500"], errors="coerce")
prices["IBM"] = pd.to_numeric(prices["IBM"], errors="coerce")
prices["1-month Tbill"] = pd.to_numeric(prices["1-month Tbill"], errors="coerce")
prices = prices.dropna(subset=["SP500", "IBM", "1-month Tbill"]).reset_index(drop=True)

## Monthly returns and risk-free rate

- `r_M`: simple monthly market return, calculated from SP 500 price series using `pct_change()` 
- `r_IBM`: simple monthly stock return for IBM  
- `r_f`: monthly risk-free rate (dividing the 1-month T-bill rate by 100)

We drop the first row and any remaining rows with missing values to create clean data dataframe `df`

In [39]:
# simple monthly returns
prices["r_M"] = prices["SP500"].pct_change()      # market
prices["r_IBM"] = prices["IBM"].pct_change()      # stock

# T-bill given is multiplied by 100 and not expressed in yearly basis
prices["r_f"] = prices["1-month Tbill"] / 100.0

# drop first row and any NaNs
df = prices.dropna(subset=["r_IBM", "r_M", "r_f"]).reset_index(drop=True)

## Excess returns and CAPM regressors

Define the excess returns for the stock and the market:

$$
r_{IBM} - r_{f} \quad \text{and} \quad r_{M} - r_{f},
$$

where
- $r_{IBM}$ is the return on IBM at time $t$,
- $r_{M}$ is the market (S\&P 500) return at time $t$,
- $r_{f}$ is the risk-free rate (1-month T-bill) at time $t$.


For different sensitivities in up and down markets, we define the indicator

$$
D_t =
\begin{cases}
1, & \text{if } r_{M} - r_{f} > 0 \quad (\text{up market}), \\
0, & \text{if } r_{M} - r_{f} \le 0 \quad (\text{down / non-up market}).
\end{cases}
$$

The regressors in Model (2) 
$$
D_t(r_{M} - r_{f}), \qquad
(1-D_t)(r_{M} - r_{f}), \qquad
(r_{M} - r_{f})^2.
$$


- $D_t = 1$ if $r_M - r_f > 0$ (up market), and $D_t = 0$ otherwise (down / non-up market).
- $D_t (r_M - r_f)$: market excess return in **up markets** ($\beta_1$).
- $(1-D_t)(r_M - r_f)$: market excess return in **down markets** ($\beta_2$).
- $(r_M - r_f)^2$: squared market excess return, capturing possible **nonlinear** effects ($\beta_3$).



In [40]:
# excess returns & 2 regressors
df["Excess_R_IBM"] = df["r_IBM"] - df["r_f"]
df["Excess_R_M"] = df["r_M"] - df["r_f"]

# indicator: up vs down markets based on market excess return
Dt = (df["Excess_R_M"] > 0).astype(int)

# beta 1,2,3
df["X1_U_M"] = Dt * df["Excess_R_M"]          
df["X2_D_M"] = (1 - Dt) * df["Excess_R_M"] 
df["X3_Squared"] = df["Excess_R_M"] ** 2       

print("Regression data:")
print(df[["Excess_R_IBM", "Excess_R_M", "X1_U_M", "X2_D_M", 
          "X3_Squared"]].head(), "\n")

Regression data:
   Excess_R_IBM  Excess_R_M   X1_U_M    X2_D_M  X3_Squared
0     -0.013152    0.014650  0.01465  0.000000    0.000215
1     -0.009519   -0.001003 -0.00000 -0.001003    0.000001
2     -0.149927   -0.034068 -0.00000 -0.034068    0.001161
3     -0.135733   -0.076757 -0.00000 -0.076757    0.005892
4     -0.140462   -0.118844 -0.00000 -0.118844    0.014124 



## Standard CAPM

We estimate the standard CAPM regression:

$$
r_{IBM} - r_{f} = \alpha + \beta (r_{M} - r_{f}) + u_t
$$

using OLS, with `Excess_R_IBM` as the dependent variable and `Excess_R_M` as the regressor.

- $\hat{\alpha}$: the estimated **abnormal return**.
- $\hat{\beta}$: IBM’s **systematic risk** relative to the market.
- The t-test for $H_0:\ \alpha = 0$ checks if the alpha is statistically different from zero.


In [41]:
# regression analysis
print("\n Model 1: Standard CAPM (OLS)")
# dependent v = Ri,t     independent v = RM,t
capm = ols("Excess_R_IBM ~ Excess_R_M", data=df).fit()
print(capm.summary())

# t-test: H0: α = 0 in model 1
print("\n t-Test for H_0: alpha = 0")
t_stat = capm.tvalues["Intercept"]
p_val = capm.pvalues["Intercept"]

print(f"t-statistic: {t_stat:.5f}")
print(f"P-value: {p_val:.5f}")

if p_val < 0.05:
    print("Decision: We reject H_0. Alpha is different from zero at 5% significance level.")
else:
    print("Decision: Fail to reject H_0. Alpha is not statistically different from zero at 5% significance level.")



 Model 1: Standard CAPM (OLS)
                            OLS Regression Results                            
Dep. Variable:           Excess_R_IBM   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     138.4
Date:                Tue, 25 Nov 2025   Prob (F-statistic):           4.24e-27
Time:                        23:29:39   Log-Likelihood:                 543.44
No. Observations:                 347   AIC:                            -1083.
Df Residuals:                     345   BIC:                            -1075.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0027

### Interpretation of the t-test for $H_0 : \alpha = 0$

From Model 1 we get:
- t-statistic for $\alpha$: approximately $0.99$,
- p-value: approximately $0.32$.

At the 5% significance level, the p-value is larger than 0.05, so that we **fail to reject** the null hypothesis

Under the observation of the sample, there is no statistical evidence that IBM’s alpha is
different from zero.




## Extended Asymmetric CAPM

Extend CAPM so that we have different betas in up and down markets and a possible nonlinear effect

$$
r_{IBM} - r_f
= \alpha
+ \beta_1 D_t (r_M - r_f)
+ \beta_2 (1-D_t)(r_M - r_f)
+ \beta_3 (r_M - r_f)^2
+ u_t .
$$


- $D_t = 1$ if $r_M - r_f > 0$ (up market), and $D_t = 0$ otherwise (down / non-up market).
- The regressors: $D_t(r_M - r_f)$, $(1-D_t)(r_M - r_f)$ and $(r_M - r_f)^2$.
- $\beta_1$ is the **up-market beta**, $\beta_2$ is the **down-market beta**, and $\beta_3$ captures **nonlinear** effects of the market excess return.

We estimate this model by OLS using `Excess_R_IBM` as the dependent variable and
`X1_U_M`, `X2_D_M`, `X3_Squared` as the regressors.


In [42]:
print("\n Model 2: Extended Asymmetric Model (OLS)")
# 3 variables β (1,2,3)
extended_model = ols("Excess_R_IBM ~ X1_U_M + X2_D_M + X3_Squared", 
                     data=df).fit()
print(extended_model.summary())

# hyphotesis test
# F-Test: H0: β1 = β2 in model 2
print("\n F-Test for H_0: beta_1 = beta_2")
f_test_result = extended_model.f_test("X1_U_M = X2_D_M")
F_stat = float(f_test_result.fvalue)
F_pval = float(f_test_result.pvalue)

print(f"F-statistic: {F_stat:.5f}")
print(f"P-value: {F_pval:.5f}")

if F_pval < 0.05:
    print("Decision: We reject H_0. The up and down market betas are significantly different under the observed data.")
else:
    print("Decision: We fail to reject H_0. No significant difference in betas under the observed data.")


 Model 2: Extended Asymmetric Model (OLS)
                            OLS Regression Results                            
Dep. Variable:           Excess_R_IBM   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.293
Method:                 Least Squares   F-statistic:                     48.87
Date:                Tue, 25 Nov 2025   Prob (F-statistic):           2.53e-26
Time:                        23:29:39   Log-Likelihood:                 546.66
No. Observations:                 347   AIC:                            -1085.
Df Residuals:                     343   BIC:                            -1070.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept

### Interpretation of the F-test for $H_0 : \beta_1 = \beta_2$

In Model 2 we test
$$
H_0 : \beta_1 = \beta_2
\quad \text{vs} \quad
H_1 : \beta_1 \neq \beta_2,
$$

Check whether IBM beta is the same in up and down markets.

A small p-value (below 5%) leads us to reject $H_0$ and conclude that the up-market and down-market betas are **significantly different** under the observed data.

If the p-value is above 5%, we will fail to reject $H_0$ and there is **no statistical evidence**
that the betas will be different during up and down markets.
